### Libraries, paths, and set-up
This notebook makes usage of particular functions found in src. These are to process .xlsx files and merge multiple year-files into one dataframe

In [1]:
import pandas as pd
import datetime as dt
import glob
import os
os.chdir('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia')
import urllib.request    
from src.data.processing import *
from src.utils.utils import *

if not os.path.exists('data/interim'):
    os.mkdir('data/iterim')

In [2]:
data_path = 'data/external/'
output_path = 'data/interim/'
supplyprice_path = 'prices/supplyprice/'
capacity_path = 'capacity/'
demand_path = 'demand/'
availability_path = 'availability/'
resource_path = 'resource/'
stockprice_path = 'prices/stockprice/national_stock/'
generation_path = 'generation/'
marginalcost_path = 'prices/marginalcost'

In [3]:
tech_dict = {
    'TERMICA': 'Thermal',
    'EOLICA': 'Wind',
    'HIDRAULICA': 'Hydro',
    'SOLAR': 'Solar',
    'COGENERADOR':'Thermal'
}

stat_dict = {
    'OPERACIÓN':'Active',
    'PRUEBAS':'Trials',
    'INACTIVO':'Inactive',
    'TRAMITE':'Construction'
}

fuel_dict = {
    'BAGAZO': 'Bagasse',
    'BIOGAS': 'Biogas',
    'AGUA': 'Water',
    'RAD SOLAR': 'Solar Radiation',
    'VIENTO':'Wind',
    'CARBON':'Coal',
    'GAS':'Gas',
    'ACPM':'Diesel',
    'DIESEL MAR':'Marine Diesel'
}

class_dict = {
    'AUTOGENERADOR':'Autogenerator', 
    'AUTOG PEQ. ESCALA':'Autogenerator', 
    'NORMAL':'Normal', 
    'GEN. DISTRIBUIDA':'Distributed',
    'FILO DE AGUA':'Run-of-river',
    'CICLO COMBINADO':'Combined Cycle',
    'FILO AGUA ESPECIAL':'Run-of-river',
    'COGENERADOR':'Cogenerator'
}

### DataFrames:
All XM/Sinergox information comes from their official website for energy market data: https://sinergox.xm.com.co/Paginas/Home.aspx

In [4]:
# Hourly Commercial Demand:
demand = process_excel(data_path, demand_path, 'h_demand.csv', ['date','distributor_code'], usecols=list(range(26)), column_modifications=[nan_filler])
demand = melt_dataframe(demand, ['date','distributor_code'],'hour','demand_hourly')
demand.to_csv(output_path + 'h_demand.csv', index=False)


# Hourly Stock Price:
hourlystockprice = process_excel(data_path, stockprice_path, 'h_stockprice.csv', ['date'], usecols=list(range(25)), nrows=366, column_modifications=[nan_filler])
hourlystockprice = melt_dataframe(hourlystockprice, ['date'], 'hour', 'hourly_bid')
hourlystockprice.to_csv(output_path + 'h_stockprice.csv', index=False)

# Hourly Supply:
availability = process_excel(data_path, availability_path, 'h_supply.csv', ['date','plant','agent_code'], usecols=list(range(27)), column_modifications=[replace_plant_names, nan_filler])
availability = melt_dataframe(availability, ['date','plant','agent_code'],'hour','supply_hourly')
availability.to_csv(output_path + 'h_supply.csv', index=False)

# Hourly Generation:
generation = process_excel(data_path, generation_path, 'h_generation.csv', ['date','plant','technology','fuel','agent_code',
                                                                            'dispatch_type','0','1','2','3','4','5','6','7','8','9','10',
                                                                            '11','12','13','14','15','16','17','18','19','20','21','22','23'],
                                                                            usecols=list(range(30)), column_modifications=[replace_plant_names, nan_filler])

generation = melt_dataframe(generation, ['date','plant','technology','fuel','agent_code','dispatch_type'],
                            'hour','generation_hourly')

generation['fuel'] = generation['fuel'].replace(fuel_dict)
generation['technology'] = generation['technology'].replace(tech_dict)
generation['dispatch_type'] = generation['dispatch_type'].replace({'DESPACHADO CENTRALMENTE':'DC',
                                                                  'NO DESPACHADO CENTRALMENTE':'ND'})
generation.to_csv(output_path + 'h_generation.csv', index=False)

In [5]:
# Hourly Marginal Cost:
marginalcost = process_excel(data_path,marginalcost_path,'h_marginalcost.csv',['date','0','1','2','3','4','5','6','7','8','9','10',
                                                                           '11','12','13','14','15','16','17','18','19','20','21','22','23','f','g'],
                                                                           usecols=list(range(25)), column_modifications=[back_filler])

marginalcost = marginalcost.drop(columns=['f','g'])
marginalcost = melt_dataframe(marginalcost, ['date'], 'hour', 'hourly_mc')
marginalcost = mcost_filler(marginalcost)
marginalcost.to_csv(output_path + 'h_marginalcost.csv', index=False)

/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/src/data/processing.py:133: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method='bfill')
/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/src/data/processing.py:155: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['hourly_mc'] = df['hourly_mc'].fillna(method='ffill')


In [6]:
# Daily Prices (Supply & Stock) by Fuel:
pricexfuel = pd.read_excel(data_path+'prices/'+'by_fuel_type.xlsx',
                              header=0,
                              decimal=',')

pricexfuel = pricexfuel.rename(columns={'Precio Oferta Ponderado':'wa_ask',
                                         'Precio Bolsa Nacional Ponderado':'wa_bid',
                                         'Fuente':'fuel',
                                         'Fecha':'date'})

fuel_dict = {
    'Carbón':'Coal', 
    'Agua':'Water', 
    'Líquidos':'Liquids', 
    'Mezcla':'Mix',
    'Gas':'Gas',
    'Orgánico':'Organic'
}

pricexfuel['fuel'] = pricexfuel['fuel'].replace(fuel_dict)
pricexfuel = nan_filler(pricexfuel)
pricexfuel.to_csv(output_path + 'd_pricexfuel.csv', index=False)

In [43]:
# ENSO Index:
url = 'https://www.cpc.ncep.noaa.gov/data/indices/soi'
urllib.request.urlretrieve(url, 'data/external/soi/soi.txt')

soi = pd.read_csv(url, skiprows=87, delim_whitespace=True,
                 skipfooter=8, engine='python')

soi = pd.melt(soi, id_vars='YEAR', var_name='MONTH')
soi = soi.rename(columns={'value': 'soi'})

soi['date'] = pd.to_datetime(soi['YEAR'].astype(str) + '-' + soi['MONTH'], format='%Y-%b')
soi['date'] = soi['date'].dt.strftime('%Y-%m')


soi = soi.drop(['YEAR', 'MONTH'], axis=1)
soi = soi.sort_values('date').reset_index(drop=True)
soi = soi[708:]
soi = soi.reset_index(drop=True)
soi.set_index('date', inplace=True) # Setting date as index
soi.index = pd.to_datetime(soi.index) #In order to forward-fill successfully

idx = pd.date_range(start=soi.index.min(), end='2022-12-31') # Creating a daily range 
soi = soi.reindex(idx).ffill() # Reindexing for daily entries and forward filling

soi.reset_index(inplace=True)
soi = soi.rename(columns={'index': 'date'})
soi['month'] = soi['date'].dt.strftime('%Y-%m')
soi = nan_filler(soi)
soi.to_csv(output_path + 'soi.csv',index=False)

In [8]:
# Power Plant (Resource) Information:

resource = pd.read_excel(data_path + resource_path + 'Listado_Recursos_Generacion.xlsx',
                      header=4,
                      decimal=',',
                      names=['SIC_code','plant','netcapacity_MW','heat_rate','is_minor','dispatch_type','fuel',
                      'date_opened','town','department','agent','status','technology','class'],
                      index_col=False)

resource = resource.drop([336,322])

resource['town'] = resource['town'].replace({'PTO. LIBERTADOR':'PUERTO LIBERTADOR',
                                                       'PTO NARE': 'PUERTO NARE',
                                                       'CALIMA (DARIEN)': 'CALIMA'})
 
resource['plant'] = resource['plant'].replace({'FLORES I CC':'FLORES 1 CC',
                                               'COGENERADOR PROENCA':'COGENERADOR PROENCA 1',
                                               'PROENCA II':'COGENERADOR PROENCA 2'})

resource['technology'] = resource['technology'].replace(tech_dict)
resource['status'] = resource['status'].replace(stat_dict)
resource['fuel'] = resource['fuel'].replace(fuel_dict)
resource['class'] = resource['class'].replace(class_dict)
resource['netcapacity_kW'] = resource['netcapacity_MW']*1000
resource = resource.drop(columns=['netcapacity_MW'])

resource = nan_filler(resource)
resource.to_csv(output_path + 'resource.csv', index=False)

In [9]:
# Daily Supply Price:
supplyprice_modifications = [replace_plant_names, nan_filler]
supplyprice = process_excel(data_path, supplyprice_path, 'd_supplyprice.csv', ['date', 'plant', 'agent_code','ideal_ask','dispatch_ask','daily_ask'],
                            usecols=list(range(6)), column_modifications=supplyprice_modifications)

supplyprice.to_csv(output_path + 'd_supplyprice.csv', index=False)

# Daily Net Capacity:
capacity_modifications = [modify_capacity, nan_filler]
capacity = process_files(data_path, output_path, capacity_path, 'd_netcapacity.csv', ['date','capacity_MW','technology'], capacity_modifications)
